In [1]:
import numpy as np
import matplotlib.pyplot as plt

import time
import os
import struct

from ymaeda_tools.read_snapshot import extract_snapshot_1d, extract_snapshot

df = 0.002441406 # frequency step size used by YMAEDA_TOOLS
f = np.arange(0, df * 2049, df) # frequency half space
F = np.arange(0, df * 4096, df) # frequency full space

In [2]:
#HDD_LOC = "/Users/ynatsume/jupyter_main/Shinmoedake Data/" 
HDD_LOC = "/Volumes/TOSHIBA_HDD/Final Year Projects/Shinmoedake Data/EIC_BACKUP/home/natsume/"
GREENS_FUNCTION = "GFpow34"

In [3]:
# Observation station names.
stations = ["SMN", "SMW"]

# Green functions impulse directions.
impulses = ["_EW_SMALL", "_NS_SMALL", "_UD_SMALL"]

# Observed motion components.
components = ["x", "y", "z"]

In [4]:
# Candidate source location.

x_src = -10900
y_src = -121100
z_src = 1000

In [5]:
STATION = stations[0] + impulses[0]
snapshot_dir = os.path.join(HDD_LOC, GREENS_FUNCTION, STATION, "PML/snapshot")

In [6]:
# Green's functions time domain parameters.

t0 = 0.0
t1 = 12.0
dt = 0.1

n_steps = int((t1 - t0) / dt) + 1 # number of time steps
g = np.zeros(n_steps) # final output green functions in t; x, y, z
t = np.arange(t0, t1 + dt, dt) # final output times 

BYTELEN = 8 # snapshot .3db file byte length

In [11]:
# Check if the spatial parameters are consistent across all stations and impulses.

for s in stations:
    for i in impulses:
        for k in components:
            stn = s + i
            snap_dir = os.path.join(HDD_LOC, GREENS_FUNCTION, stn, "PML/snapshot")
            snapshot_file = os.path.join(snapshot_dir, "source.F" + k + ".t" + "0.0000" + ".3db")
            with open(snapshot_file, mode = "rb") as File:
                fileContent = File.read()
                N = struct.unpack("iii", fileContent[0:12])
                x0 = struct.unpack("ddd", fileContent[12:36])
                dx = struct.unpack("ddd", fileContent[36:60])
            print(os.path.join(GREENS_FUNCTION, stn, "source.F" + k + ".t" + "0.0000" + ".3db"))
            print(N, x0, dx)

GFpow34/SMN_EW_SMALL/source.Fx.t0.0000.3db
(161, 160, 140) (-11800.0, -121495.0, 5.0) (10.0, 10.0, 10.0)
GFpow34/SMN_EW_SMALL/source.Fy.t0.0000.3db
(160, 161, 140) (-11795.0, -121500.0, 5.0) (10.0, 10.0, 10.0)
GFpow34/SMN_EW_SMALL/source.Fz.t0.0000.3db
(160, 160, 141) (-11795.0, -121495.0, 0.0) (10.0, 10.0, 10.0)
GFpow34/SMN_NS_SMALL/source.Fx.t0.0000.3db
(161, 160, 140) (-11800.0, -121495.0, 5.0) (10.0, 10.0, 10.0)
GFpow34/SMN_NS_SMALL/source.Fy.t0.0000.3db
(160, 161, 140) (-11795.0, -121500.0, 5.0) (10.0, 10.0, 10.0)
GFpow34/SMN_NS_SMALL/source.Fz.t0.0000.3db
(160, 160, 141) (-11795.0, -121495.0, 0.0) (10.0, 10.0, 10.0)
GFpow34/SMN_UD_SMALL/source.Fx.t0.0000.3db
(161, 160, 140) (-11800.0, -121495.0, 5.0) (10.0, 10.0, 10.0)
GFpow34/SMN_UD_SMALL/source.Fy.t0.0000.3db
(160, 161, 140) (-11795.0, -121500.0, 5.0) (10.0, 10.0, 10.0)
GFpow34/SMN_UD_SMALL/source.Fz.t0.0000.3db
(160, 160, 141) (-11795.0, -121495.0, 0.0) (10.0, 10.0, 10.0)
GFpow34/SMW_EW_SMALL/source.Fx.t0.0000.3db
(161, 160, 1

In [ ]:
for n in range(n_steps): # loop over all time steps. One file for one time step...
    TIME_ZEROS = format(t0 + dt * n, "0.4f")
    snapshot_file = os.path.join(snapshot_dir, "source.F" + k + ".t" + TIME_ZEROS + ".3db")
            
    with open(snapshot_file, mode = "rb") as File:
        fileContent = File.read()
        if n == 0:
            N = struct.unpack("iii", fileContent[0:12]) # number of elements per axis
            x0 = struct.unpack("ddd", fileContent[12:36]) # starting point of each axis
            dx = struct.unpack("ddd", fileContent[36:60]) # distance step size per axis
            data_length = int((len(fileContent) - 60) / BYTELEN)
                
        # From the 3D snapshot data, extract only the data for the wanted point [X, Y Z].
        # Convert the geographical coordinates to indices of the binary data:
        idx, idy, idz = snapshot_stnloc(N, x0, dx, x_src, y_src, z_src)
        
        # We have to perform spatial derivatives of the Green's functions with respect
        # to x, y and z. Therefore instead of actually loading the snapshot at the 
        # specified candidate location, we load the snapshot time series for the grid
        # points surrounding the candidate location.
        
        counter = index_convert31(idx, idy, idz, N[0], N[1], N[2])
        L = 60 + BYTELEN * counter # Left most index of the wanted data.
        R = L + BYTELEN            # Right most index of the wanted data.
        g[n] = struct.unpack("d", fileContent[L:R])[0]